In [1]:
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline

RuntimeError: Polyfit sanity test emitted a warning, most likely due to using a buggy Accelerate backend. If you compiled yourself, see site.cfg.example for information. Otherwise report this to the vendor that provided NumPy.
RankWarning: Polyfit may be poorly conditioned


In [ ]:
from pymongo import MongoClient
import configparser

class Mongo():

    def __init__(self, config_file):
        config = configparser.RawConfigParser()
        config.read(config_file)

        self.server = config.get('mongo', 'server') if 'server' in [i[0] for i in config.items('mongo')] else False
        self.db = config.get('mongo', 'db') if 'db' in [i[0] for i in config.items('mongo')] else False
        self.host = config.get('mongo', 'host') if 'host' in [i[0] for i in config.items('mongo')] else False
        self.port = int(config.get('mongo', 'port')) if 'port' in [i[0] for i in config.items('mongo')] else False
        self.user = config.get('mongo', 'user') if 'user' in [i[0] for i in config.items('mongo')] else False
        self.password = config.get('mongo', 'password') if 'password' in [i[0] for i in config.items('mongo')] else False

        print(self.server)
        
    def connect(self):
        if self.server == 'True':
            connection_string_basic = "mongodb+srv://{user}:{password}@{host}{port}{db}".format(user=self.user,
                                                                                                   password=self.password,
                                                                                                   host=self.host,
                                                                                                   port="{port}",
                                                                                                   db="{db}")
        else:
            connection_string_basic = "mongodb://{user}:{password}@{host}{port}{db}".format(user=self.user,
                                                                                            password=self.password,
                                                                                            host=self.host,
                                                                                            port="{port}",
                                                                                            db="{db}")

        if self.port:
            connection_string_port = connection_string_basic.format(port=":{}".format(self.port),
                                                                    db="{db}")
        else:
            connection_string_port = connection_string_basic.format(port="",
                                                                    db="{db}")

        if self.db:
            connection_string = connection_string_port.format(db="/{}".format(self.db))
        else:
            connection_string = connection_string_port.format(db="")
        
        print(connection_string)
        return MongoClient(connection_string)


In [ ]:
mongo = Mongo('config.cfg')
mongo_client = mongo.connect()

In [ ]:
mongo_client.datapop.tweets_to_classify.aggregate([{"$sample":{"size": 3}}])

In [ ]:
sample = {str(obj['_id']): {i:obj[i] for i in obj if i != '_id'} for obj in mongo_client.datapop.tweets_to_classify.aggregate([{'$sample': {'size': 3}}])}
tweets = {str(obj['_id']): {i:obj[i] for i in obj if i != '_id'} for obj in mongo_client.datapop.tweets_classified.find()}

In [ ]:
tweets_list = [tweets[key] for key in tweets.keys()]

In [ ]:
df_tweets = pd.DataFrame.from_dict(tweets_list)

In [ ]:
df_tweets

In [ ]:
df_tweets.sentiment.value_counts().plot.bar()

In [ ]:
df_positives = df_tweets[df_tweets['sentiment'] == 'positive'][['full_text', 'sentiment', 'rate', 'classified_by']]

In [ ]:
df_neutrals = df_tweets[df_tweets['sentiment'] == 'neutral'][['full_text', 'sentiment', 'rate', 'classified_by']]

In [ ]:
df_negatives = df_tweets[df_tweets['sentiment'] == 'negative'][['full_text', 'sentiment', 'rate', 'classified_by']]

In [ ]:
df_positives

In [ ]:
df_neutrals

In [ ]:
df_negatives

In [ ]:
df_tweets.rate.value_counts().sort_index().plot.bar()

In [ ]:
from nltk.corpus import stopwords
import nltk
import re

## Tokenizando os textos

In [ ]:
from nltk.tokenize import TweetTokenizer

tweet_tokenizer = TweetTokenizer()

In [ ]:
def collectTokens(tweets):
    list_of_word = []
    for tweet, sentiment in tweets:
        list_of_word = list_of_word + [(token.lower(), sentiment) for token in tweet_tokenizer.tokenize(tweet)]
    
    return list_of_word
        

In [ ]:
for i in df_tweets[['full_text', 'sentiment']].index:
    print(df_tweets[['full_text', 'sentiment']].iloc[i]['full_text'])

In [ ]:
tokens = collectTokens([(df_tweets[['full_text', 'sentiment']].iloc[i]['full_text'],
                         df_tweets[['full_text', 'sentiment']].iloc[i]['sentiment']) for i in df_tweets[['full_text',
                                                                                                        'sentiment']].index])

In [ ]:
len(tokens)

In [ ]:
tokens

## Removendo Stopwords

In [ ]:
portuguese_stop_words = stopwords.words('portuguese')

In [ ]:
def removeStopwords(tokens):
    return [token for token in tokens if token[0] not in portuguese_stop_words + ['pra', 'vc', 'q', 'vcs']]

In [ ]:
tokens_no_stopwords = removeStopwords(tokens)

In [ ]:
len(tokens_no_stopwords)

In [ ]:
tokens_no_stopwords

## Removendo Menções

In [ ]:
def removeMentions(tokens):
    mentions_regex = re.compile('[^@]')
    return list(filter(lambda tweet: mentions_regex.match(tweet[0]), tokens))

In [ ]:
tokens_no_mentions = removeMentions(tokens_no_stopwords)

In [ ]:
len(tokens_no_mentions)

In [ ]:
tokens_no_mentions

## Removendo links

In [ ]:
def removeLinks(tokens):
    links_regex = re.compile('^(?!http).*')
    return list(filter(lambda tweet: links_regex.match(tweet[0]), tokens))

In [ ]:
tokens_no_links = removeLinks(tokens_no_mentions)

In [ ]:
len(tokens_no_links)

In [ ]:
tokens_no_links

## Removendo Números e placares

In [ ]:
def removeNumbersAndScores(tokens):
    numbers_regex = re.compile('(([0-9]x[0-9])|(^x$)|(^[0-9]+$))')
    return [token for token in tokens if token not in list(filter(lambda tweet: numbers_regex.match(tweet[0]), tokens))]

In [ ]:
tokens_no_numbers_scores = removeNumbersAndScores(tokens_no_links)

In [ ]:
len(tokens_no_numbers_scores)

In [ ]:
tokens_no_numbers_scores

## Remove Pontuação

In [ ]:
def removePontuation(tokens):
    pontuation_regex = re.compile('[^!\.?:\[\],\{\}\\\/;"\(\)]')
    return list(filter(lambda tweet: pontuation_regex.match(tweet[0]), tokens))

In [ ]:
tokens_no_pontuation = removePontuation(tokens_no_numbers_scores)

In [ ]:
len(tokens_no_pontuation)

In [ ]:
tokens_no_pontuation

## Removendo Nome de Times

In [ ]:
def removeTeams(tokens):
    teams_list = ['fluminense',
                  'flamengo',
                  'botafogo',
                  'vasco',
                  'são paulo',
                  'palmeiras',
                  'santos',
                  'corinthians',
                  'cruzeiro',
                  'atlético mg',
                  'atlético mineiro',
                  'atletico mg',
                  'atletico mineiro',
                  'internacional',
                  'gremio',
                  'chapecoense',
                  'avai',
                  'csa',
                  'bahia',
                  'goias',
                  'athletico pr',
                  'athletico paranaense',
                  'atletico paranaense',
                  'atlético paranaense'
                  'ceara',
                  'fortaleza'
                 ]
    return [token for token in tokens if token[0] not in teams_list]

In [ ]:
token_no_teams = removeTeams(tokens_no_pontuation)

In [ ]:
len(token_no_teams)

In [ ]:
token_no_teams

## Stemming

In [ ]:
from nltk.stem import RSLPStemmer

In [ ]:
def stem(tokens):
    stemmer = RSLPStemmer()
    
    return [(stemmer.stem(token[0]), token[1]) for token in tokens]

In [ ]:
token_stemmed = stem(token_no_teams)

## Dataframe de Tokens

In [ ]:
df_token = pd.DataFrame(token_no_teams, columns =['token', 'sentiment']) 

In [ ]:
df_token['token'].describe()

In [ ]:
df_token_grouped = df_token.groupby(['token', 'sentiment']).size().unstack(fill_value=0)

In [ ]:
df_token_grouped['total'] = df_token_grouped['positive']\
                            + df_token_grouped['neutral']\
                            + df_token_grouped['negative']

In [ ]:
df_token_grouped.iloc[1000:1300]

In [ ]:
df_token_grouped.sort_values('total', ascending=False)[['positive', 'neutral', 'negative']].iloc[:20].plot.bar(figsize=(15,10))

In [ ]:
df_token_grouped.sort_values('positive', ascending=False)[['positive', 'neutral', 'negative']].iloc[:20].plot.bar(figsize=(15,10))

In [ ]:
df_token_grouped.sort_values('neutral', ascending=False)[['positive', 'neutral', 'negative']].iloc[:20].plot.bar(figsize=(15,10))

In [ ]:
df_token_grouped.sort_values('negative', ascending=False)[['positive', 'neutral', 'negative']].iloc[:20].plot.bar(figsize=(15,10))

In [ ]:
df_token_grouped['sentiment'] = df_token_grouped.apply(lambda r: 'positive' if r.max() == r.positive 
                                                       else 'negative' if r.max() == r.negative
                                                       else 'neutral',
                                                       axis=1)
#df_token_grouped.loc['#'].max()

In [ ]:
df_token_grouped

In [ ]:
## Dataframe de Tokens

df_token = pd.DataFrame(token_stemmed, columns =['token', 'sentiment']) 

df_token['token'].describe()

df_token_grouped = df_token.groupby(['token', 'sentiment']).size().unstack(fill_value=0)

df_token_grouped['total'] = df_token_grouped['positive']\
                            + df_token_grouped['neutral']\
                            + df_token_grouped['negative']

df_token_grouped['sentiment'] = df_token_grouped.apply(lambda r: 'positive' if r.max() == r.positive 
                                                       else 'negative' if r.max() == r.negative
                                                       else 'neutral',
                                                       axis=1)

df_token_grouped.iloc[1000:1300]

df_token_grouped.sort_values('total', ascending=False)[['positive', 'neutral', 'negative']].iloc[:20].plot.bar(title='Total', figsize=(15,10))

df_token_grouped.sort_values('positive', ascending=False)[['positive', 'neutral', 'negative']].iloc[:20].plot.bar(title='Positive', figsize=(15,10))

df_token_grouped.sort_values('neutral', ascending=False)[['positive', 'neutral', 'negative']].iloc[:20].plot.bar(title='Neutral', figsize=(15,10))

df_token_grouped.sort_values('negative', ascending=False)[['positive', 'neutral', 'negative']].iloc[:20].plot.bar(title='Negative', figsize=(15,10))

In [ ]:
#def predict():
df_token_grouped

## Bag of Words    

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv =  CountVectorizer(lowercase=True,stop_words=portuguese_stop_words + ['pra', 'vc', 'q', 'vcs'],ngram_range = (1,1),tokenizer = tweet_tokenizer.tokenize)

In [ ]:
text_counts = cv.fit_transform(df_tweets['full_text'])

## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
text_tf= tf.fit_transform(df_tweets['full_text'])

## Split Train Test

### Bow

In [ ]:
from sklearn.model_selection import train_test_split
BoW_X_train, BoW_X_test, BoW_y_train, BoW_y_test = train_test_split(
    text_counts, df_tweets['sentiment'], test_size=0.3, random_state=1)

### TF-IDF

In [ ]:
from sklearn.model_selection import train_test_split
TF_X_train, TF_X_test, TF_y_train, TF_y_test = train_test_split(
    text_tf, df_tweets['sentiment'], test_size=0.3, random_state=1)

## MultinomialNB 

### BoW

In [ ]:
from sklearn.naive_bayes import MultinomialNB
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Generation Using Multinomial Naive Bayes
clf = MultinomialNB().fit(BoW_X_train, BoW_y_train)
predicted= clf.predict(BoW_X_test)
print("MultinomialNB Accuracy:",metrics.accuracy_score(BoW_y_test, predicted))

### TF-IDF

In [ ]:
from sklearn.naive_bayes import MultinomialNB
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Generation Using Multinomial Naive Bayes
clf = MultinomialNB().fit(TF_X_train, TF_y_train)
predicted= clf.predict(TF_X_test)
print("MultinomialNB Accuracy:",metrics.accuracy_score(TF_y_test, predicted))

In [ ]:
clf.predict("testando")

In [ ]:
def removeLinks(tokens):
    links_regex = re.compile('^(?!http).*')
    return list(filter(lambda tweet: links_regex.match(tweet[0]), tokens))

tokens_no_links = removeLinks(tokens_no_mentions)


